# Validation with SSAM Results 

We'll compare the results obtained by my SSM code and the output of the SSAM software

In [12]:
import sys

import numpy as np
import pandas as pd

In [13]:
sys.path.append(
    r'C:\Users\fvall\Documents\Research\TrafficSimulation\SSMEstimation')
import readers # necessary to reload the module later if needed
from classesForSSM import SSMEstimator
# import readers

Increase cell length

In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Load my Data and Compute SSM

In [38]:
network_name = 'I710-MultiSec-3mi'#'highway_in_and_out_lanes' #'I710-MultiSec-3mi'

In [39]:
reader = readers.PostProcessedDataReader(network_name)

In [40]:
my_data = reader.load_data()

In [41]:
ssm_estimator = SSMEstimator(my_data)

In [42]:
ssm_estimator.include_ttc()

In [43]:
my_data.head()

,time,veh_id,veh_type,lane,x,vx,y,leader_id,delta_x,delta_v,leader_type,TTC
0,582.6,11,100,3,1565.21,28.080556,0.5,11,250.00,0.000,0.0,inf
1,582.6,17,200,2,1371.05,27.922222,0.5,29,179.70,-4.550,100.0,inf
2,582.6,18,100,3,1042.09,27.350000,0.5,18,250.00,0.000,0.0,inf
3,582.6,21,100,3,559.36,26.555556,0.5,21,250.00,0.000,0.0,inf
4,582.6,23,100,2,493.85,26.497222,0.5,40,42.58,-1.775,100.0,inf


### Compute TTC from vissim speed diff 

In [32]:
my_data['vissim_ttc'] = my_data['delta_x']/my_data['vissim_delta_v']

In [33]:
my_data.loc[my_data['vissim_delta_v']<0, 'vissim_ttc'] = float('inf')

In [34]:
my_data.head()

,time,veh_id,veh_type,link,lane,x,vx,y,leader_id,delta_x,vissim_delta_v,delta_v,leader_type,TTC,vissim_ttc
0,79.5,13,100,10003,1,98.79,19.341667,0.5,13,250.00,0.00,0.000000,0.0,inf,inf
1,79.5,15,100,10003,1,70.83,18.638889,0.5,13,27.89,-2.59,-0.702778,100.0,inf,inf
2,79.5,26,100,6,1,7.11,12.211111,0.5,37,25.60,-17.45,-4.783333,100.0,inf,inf
3,79.5,30,100,3,1,509.11,10.375000,0.5,15,76.34,-29.88,-8.263889,100.0,inf,inf
4,79.5,31,100,6,2,109.37,21.575000,0.5,31,250.00,0.00,0.000000,0.0,inf,inf


## Double check post-processing 

Must rerun to VISSIM simulation and save delta_v

In [10]:
reader_raw = readers.VissimDataReader(sim_name=network_name)

In [11]:
reader_raw.column_names

['time',
 'veh_id',
 'veh_type',
 'link',
 'lane',
 'x',
 'vx',
 'y',
 'leader_id',
 'delta_x']

In [ ]:
data_raw = reader_raw.load_data()

In [ ]:
data_raw.head()

## Load Results from SSAM and Check Results

In [22]:
# Copied from SSAM interface (not available in the csv file)
ttc_threshold = 1.5

In [10]:
ssam_data = pd.read_csv('I710-MultiSec-3mi_001_summary.csv')

In [11]:
split_rows = ssam_data['Summary Group'].str.strip()=='Summary Group'
first_rows = split_rows[split_rows].index.values+1
df_list = []
for i in range(len(first_rows)-1):
    last_row = first_rows[i+1] - 2
    df_list.append(ssam_data.loc[first_rows[i]:last_row])
df_list.append(ssam_data.loc[first_rows[i+1]:])

In [12]:
summary_df = df_list[-1].copy()
summary_df.columns = ssam_data.loc[first_rows[-1]-1].values
summary_df

,Summary Group,Total,unclassified,crossing,rear end,lane change,NaN
23,Unfiltered-All Files,5903,0,10,5339,554,NaN
24,Unfiltered-C:\Users\fvall\Documents\Research\T...,5903,0,10,5339,554,NaN


In [13]:
ssam_conflicts = pd.read_csv('I710-MultiSec-3mi_001.csv')
print(ssam_conflicts.shape)
ssam_conflicts.head()

(5903, 45)


,trjFile,tMinTTC,xMinPET,yMinPET,zMinPET,TTC,PET,MaxS,DeltaS,DR,...,SecondVMinTTC,SecondDeltaV,xSecondCSP,ySecondCSP,xSecondCEP,ySecondCEP,P(UEA),mTTC,mPET,Unnamed: 44
0,I710 - MultiSec - 3mi_001.trj,14.2,2512.22,13420.03,0.0,0.0,0.0,29.64,1.27,-5.33,...,28.40,0.32,2518.49,13442.00,2512.65,13427.02,1.0,99.0,99.0,NaN
1,I710 - MultiSec - 3mi_001.trj,117.2,2515.13,13420.05,0.0,0.0,0.0,25.78,1.58,-5.60,...,25.74,0.36,2522.25,13432.37,2518.67,13423.19,1.0,99.0,99.0,NaN
2,I710 - MultiSec - 3mi_001.trj,197.4,2510.96,13418.99,0.0,0.0,0.0,29.09,2.03,-5.43,...,27.42,0.56,2522.99,13443.92,2515.66,13425.10,1.0,99.0,99.0,NaN
3,I710 - MultiSec - 3mi_001.trj,235.8,2503.35,13390.79,0.0,0.0,0.0,29.67,2.48,-6.52,...,29.61,1.42,2515.10,13430.87,2501.96,13392.68,1.0,99.0,99.0,NaN
4,I710 - MultiSec - 3mi_001.trj,265.6,572.29,7744.22,0.0,0.0,0.0,28.17,3.46,-6.67,...,27.44,1.79,595.64,7774.85,574.18,7743.50,1.0,99.0,99.0,NaN


## Compare 

In [44]:
np.sum(my_data['TTC'] < ttc_threshold)

435

In [45]:
# my_data[my_data['TTC'] < ttc_threshold].head()
my_conflicts = my_data[my_data['TTC'] < ttc_threshold].groupby(['veh_id']).mean()
print(my_conflicts.shape)
my_conflicts.head()

(125, 11)


,time,veh_type,lane,x,vx,y,leader_id,delta_x,delta_v,leader_type,TTC
veh_id,,,,,,,,,,,
407,793.40,100.0,1.0,800.276000,20.035000,0.5,414.000000,7.798000,6.647222,100.0,1.188752
416,805.40,100.0,2.0,771.250000,9.366667,0.1,445.000000,8.380000,6.100000,100.0,1.373770
446,806.92,100.0,1.0,774.798000,11.512500,0.5,416.000000,7.542000,6.688333,100.0,1.047860
502,828.20,100.0,3.0,730.560769,10.433974,0.5,508.076923,6.377692,10.420299,100.0,0.652345
504,828.30,100.0,3.0,753.465000,9.324722,0.4,488.800000,3.053000,9.306667,100.0,0.339262


In [47]:
len(np.unique(my_data.loc[my_data['TTC'] < ttc_threshold, 'veh_id']))

125

In [20]:
ssam_conflicts.columns

Index(['trjFile', 'tMinTTC', 'xMinPET', 'yMinPET', 'zMinPET', 'TTC', 'PET',
       'MaxS', 'DeltaS', 'DR', 'MaxD', 'MaxDeltaV', 'ConflictAngle',
       'ClockAngle', 'ConflictType', 'PostCrashV', 'PostCrashHeading',
       'FirstVID', 'FirstLink', 'FirstLane', 'FirstLength', 'FirstWidth',
       'FirstHeading', 'FirstVMinTTC', 'FirstDeltaV', 'xFirstCSP', 'yFirstCSP',
       'xFirstCEP', 'yFirstCEP', 'SecondVID', 'SecondLink', 'SecondLane',
       'SecondLength', 'SecondWidth', 'SecondHeading', 'SecondVMinTTC',
       'SecondDeltaV', 'xSecondCSP', 'ySecondCSP', 'xSecondCEP', 'ySecondCEP',
       'P(UEA)', 'mTTC', 'mPET', 'Unnamed: 44'],
      dtype='object')

In [24]:
ssam_conflicts.loc[(ssam_conflicts['tMinTTC']>=np.min(my_data['time']))
              & (ssam_conflicts['ConflictType']=='rear end'), 
                   ['tMinTTC', 'PET', 'TTC', 'xFirstCSP', 'xSecondCSP',
                    'xFirstCEP', 'xSecondCEP',
                    'FirstVMinTTC', 'SecondVMinTTC', 'DeltaS', 'DR', 'MaxD']].head()

,tMinTTC,PET,TTC,xFirstCSP,xSecondCSP,xFirstCEP,xSecondCEP,FirstVMinTTC,SecondVMinTTC,DeltaS,DR,MaxD
12,630.6,0.0,0.0,433.13,427.67,415.37,412.51,26.37,27.01,2.39,0.93,0.93
14,734.2,0.0,0.0,2519.45,2522.50,2508.96,2509.28,30.88,31.12,1.33,-0.87,-2.01
16,794.0,0.0,0.0,22.34,19.55,19.55,18.92,10.62,29.66,19.11,-2.30,-6.16
17,806.2,0.0,0.0,2517.96,2515.12,2509.73,2506.64,28.08,30.18,2.48,-6.57,-6.57
18,807.4,0.0,0.0,21.93,22.31,21.33,22.31,1.13,0.00,1.13,0.00,0.00


In [ ]:
(433.13 - 427.67)/(27.01-26.37)

In [ ]:
(433.13 - 427.67)

In [25]:
my_data[(my_data['time'] == 806.2) & (my_data['vx']>=30)]

,time,veh_id,veh_type,lane,x,vx,y,leader_id,delta_x,delta_v,leader_type,TTC
215296,806.2,356,100,2,1358.02,30.144444,0.50,402,39.47,-0.052778,100.0,inf
215305,806.2,375,100,2,1230.88,30.400000,0.50,373,39.41,1.241667,100.0,31.739597
215310,806.2,380,100,1,1244.30,30.063889,0.50,419,212.91,-0.519444,100.0,inf
215324,806.2,397,100,3,1159.17,30.227778,0.50,400,40.55,-0.336111,100.0,inf
215327,806.2,400,100,3,1199.79,30.563889,0.50,377,45.88,3.280556,100.0,13.985436
215329,806.2,402,100,2,1397.51,30.197222,0.50,386,49.42,1.136111,100.0,43.499267
215346,806.2,419,100,1,1457.33,30.583333,0.50,406,34.18,1.338889,100.0,25.528631
215368,806.2,441,100,2,405.09,30.019444,0.50,448,160.73,-1.577778,100.0,inf
215375,806.2,448,100,2,566.14,31.597222,0.50,431,182.64,1.691667,100.0,107.964532
215444,806.2,517,100,2,294.83,30.255556,0.50,477,47.77,0.333333,100.0,143.310000


In [29]:
my_data[(my_data['time'] == 806.2) & (my_data['veh_id'] == 373)]

,time,veh_id,veh_type,lane,x,vx,y,leader_id,delta_x,delta_v,leader_type,TTC
215304,806.2,373,100,2,1270.04,29.158333,0.5,369,42.84,-0.511111,100.0,inf
